In [1]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F

In [2]:
spark = (SparkSession.builder.appName("gaurav")
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
        .getOrCreate())
spark

:: loading settings :: url = jar:file:/home/gauravthagunna/venvs/pyspark_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gauravthagunna/.ivy2/cache
The jars for the packages stored in: /home/gauravthagunna/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2be27d16-613e-400e-9d3e-cac0503bdc84;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in

In [145]:
maxOffsetsPerTrigger = 200 # max 2000 offset per microbatch
startingTime = 1 #default : start from beginning
df = (spark.read
      .format("kafka")
      .option("kafka.bootstrap.servers","localhost:9092")
      .option("subscribe","wikimedia_data_topic")
      .option("maxOffsetsPerTrigger",maxOffsetsPerTrigger)
      .option("startingTimestamp",startingTime)
      .load()
      )
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



24/11/15 17:35:50 WARN KafkaSourceProvider: maxOffsetsPerTrigger option ignored in batch queries


In [124]:
df.limit(5).show()

24/11/15 17:11:55 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 17:11:55 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 17:11:55 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 17:11:55 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 17:11:55 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+----+--------------------+--------------------+---------+------+--------------------+-------------+
| key|               value|               topic|partition|offset|           timestamp|timestampType|
+----+--------------------+--------------------+---------+------+--------------------+-------------+
|null|[22 7B 5C 22 24 7...|wikimedia_data_topic|        0|     0|2024-11-15 12:57:...|            0|
|null|[22 7B 5C 22 24 7...|wikimedia_data_topic|        0|     1|2024-11-15 12:58:...|            0|
|null|[22 7B 5C 22 24 7...|wikimedia_data_topic|        0|     2|2024-11-15 12:58:...|            0|
|null|[22 7B 5C 22 24 7...|wikimedia_data_topic|        0|     3|2024-11-15 12:58:...|            0|
|null|[22 7B 5C 22 24 7...|wikimedia_data_topic|        0|     4|2024-11-15 12:58:...|            0|
+----+--------------------+--------------------+---------+------+--------------------+-------------+



In [110]:
df1 = df.select(df.value.cast("string")) # Convert the PySpark DataFrame to a JSON string 
json_string = df1.toJSON().collect()[0] 
  
print(json_string) 
  
# Write the JSON string to a file 
with open("/home/gauravthagunna/Desktop/projects/kafka-projects/wikimedia_processing/data_sample/data_wikimedia.json", "w") as f: 
    f.write(json_string) 

24/11/15 16:55:59 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 16:55:59 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 16:55:59 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 16:55:59 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 16:55:59 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


{"value":"\"{\\\"$schema\\\":\\\"/mediawiki/recentchange/1.0.0\\\",\\\"meta\\\":{\\\"uri\\\":\\\"https://commons.wikimedia.org/wiki/File:Florida_Tropical_House_NPS.jpg\\\",\\\"request_id\\\":\\\"0c4303be-f77f-4c51-8691-9ff1571c6298\\\",\\\"id\\\":\\\"56cf14b8-fb3e-4c2d-b96b-25e88060b96d\\\",\\\"dt\\\":\\\"2024-11-15T07:12:58Z\\\",\\\"domain\\\":\\\"commons.wikimedia.org\\\",\\\"stream\\\":\\\"mediawiki.recentchange\\\",\\\"topic\\\":\\\"codfw.mediawiki.recentchange\\\",\\\"partition\\\":0,\\\"offset\\\":1249027121},\\\"id\\\":2655168092,\\\"type\\\":\\\"edit\\\",\\\"namespace\\\":6,\\\"title\\\":\\\"File:Florida Tropical House NPS.jpg\\\",\\\"title_url\\\":\\\"https://commons.wikimedia.org/wiki/File:Florida_Tropical_House_NPS.jpg\\\",\\\"comment\\\":\\\"/* wbsetclaim-update:2||1|2 */ [[d:Special:EntityPage/P6216]]: [[d:Special:EntityPage/Q19652]], updating based on [[Template:PD-USGov-NPS]]\\\",\\\"timestamp\\\":1731654778,\\\"user\\\":\\\"BotMultichillT\\\",\\\"bot\\\":true,\\\"notify

In [22]:
df1.show(truncate=False)

24/11/15 13:06:02 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 13:06:02 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 13:06:02 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 13:06:02 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 13:06:02 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [125]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType
schema = StructType([
    StructField("$schema", StringType(), True),
    StructField("meta", StructType([
        StructField("uri", StringType(), True),
        StructField("request_id", StringType(), True),
        StructField("id", StringType(), True),
        StructField("dt", TimestampType(), True),
        StructField("domain", StringType(), True),
        StructField("stream", StringType(), True),
        StructField("topic", StringType(), True),
        StructField("partition", IntegerType(), True),
        StructField("offset", IntegerType(), True),
    ]), True),
    StructField("id", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("namespace", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("title_url", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("timestamp", IntegerType(), True),
    StructField("user", StringType(), True),
    StructField("bot", BooleanType(), True),
    StructField("notify_url", StringType(), True),
    StructField("minor", BooleanType(), True),
    StructField("patrolled", BooleanType(), True),
    StructField("length", StructType([
        StructField("old", IntegerType(), True),
        StructField("new", IntegerType(), True),
    ]), True),
    StructField("revision", StructType([
        StructField("old", IntegerType(), True),
        StructField("new", IntegerType(), True),
    ]), True),
    StructField("server_url", StringType(), True),
    StructField("server_name", StringType(), True),
    StructField("server_script_path", StringType(), True),
    StructField("wiki", StringType(), True),
])

df_parsed = df.select(
    F.from_json(df.value.cast("string"), schema).alias("parsed_value"),"topic","timestamp"
)

# Step 3: Extract fields from the parsed JSON and select the necessary columns
df_final = df_parsed.select(
    "parsed_value.$schema",
    "parsed_value.meta.uri",
    "parsed_value.meta.request_id",
    "parsed_value.meta.id",
    "parsed_value.meta.dt",
    "parsed_value.meta.domain",
    "parsed_value.meta.stream",
    "parsed_value.meta.topic",
    "parsed_value.meta.partition",
    "parsed_value.meta.offset",
    "parsed_value.id",
    "parsed_value.type",
    "parsed_value.namespace",
    "parsed_value.title",
    "parsed_value.title_url",
    "parsed_value.comment",
    "parsed_value.timestamp",
    "parsed_value.user",
    "parsed_value.bot",
    "parsed_value.notify_url",
    "parsed_value.minor",
    "parsed_value.patrolled",
    "parsed_value.length.old",
    "parsed_value.length.new",
    "parsed_value.revision.old",
    "parsed_value.revision.new",
    "parsed_value.server_url",
    "parsed_value.server_name",
    "parsed_value.server_script_path",
    "parsed_value.wiki",
    "topic","timestamp"
)
df_final.show()


24/11/15 17:12:48 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 17:12:48 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 17:12:48 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 17:12:48 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 17:12:48 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+-------+----+----------+----+----+------+------+-----+---------+------+----+----+---------+-----+---------+-------+---------+----+----+----------+-----+---------+----+----+----+----+----------+-----------+------------------+----+--------------------+--------------------+
|$schema| uri|request_id|  id|  dt|domain|stream|topic|partition|offset|  id|type|namespace|title|title_url|comment|timestamp|user| bot|notify_url|minor|patrolled| old| new| old| new|server_url|server_name|server_script_path|wiki|               topic|           timestamp|
+-------+----+----------+----+----+------+------+-----+---------+------+----+----+---------+-----+---------+-------+---------+----+----+----------+-----+---------+----+----+----+----+----------+-----------+------------------+----+--------------------+--------------------+
|   null|null|      null|null|null|  null|  null| null|     null|  null|null|null|     null| null|     null|   null|     null|null|null|      null| null|     null|null|null|null|nul

In [174]:
df_final.count()

24/11/15 18:22:57 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 18:22:57 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 18:22:57 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 18:22:57 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 18:22:57 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


182

In [177]:
df_final.filter(F.col("request_id").isNull()).count()

24/11/15 18:24:04 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 18:24:04 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 18:24:04 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 18:24:04 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 18:24:04 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


182

In [120]:

#{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://commons.wikimedia.org/wiki/File:Florida_Tropical_House_NPS.jpg","request_id":"0c4303be-f77f-4c51-8691-9ff1571c6298","id":"56cf14b8-fb3e-4c2d-b96b-25e88060b96d","dt":"2024-11-15T07:12:58Z","domain":"commons.wikimedia.org","stream":"mediawiki.recentchange","topic":"codfw.mediawiki.recentchange","partition":0,"offset":1249027121},"id":2655168092,"type":"edit","namespace":6,"title":"File:Florida Tropical House NPS.jpg","title_url":"https://commons.wikimedia.org/wiki/File:Florida_Tropical_House_NPS.jpg","comment":"/* wbsetclaim-update:2||1|2 */ [[d:Special:EntityPage/P6216]]: [[d:Special:EntityPage/Q19652]], updating based on [[Template:PD-USGov-NPS]]","timestamp":1731654778,"user":"BotMultichillT","bot":true,"notify_url":"https://commons.wikimedia.org/w/index.php?diff=957288243&oldid=709348572&rcid=2655168092","minor":false,"patrolled":true,"length":{"old":4685,"new":5358},"revision":{"old":709348572,"new":957288243},"server_url":"https://commons.wikimedia.org","server_name":"commons.wikimedia.org","server_script_path":"/w","wiki":"commonswiki","parsedcomment":"u200e<span dir="auto"><span class="autocomment">Changed claim: </span></span> <a href="https://www.wikidata.org/wiki/Special:EntityPage/P6216" class="extiw" title="u200ecopyright statusu200e | u200ecopyright status for intellectual creations like works of art, publications, software, etc.u200e"><span class="wb-itemlink"><span class="wb-itemlink-label" lang="en" dir="ltr">copyright status</span> <span class="wb-itemlink-id">(P6216)</span></span></a>: <a href="https://www.wikidata.org/wiki/Special:EntityPage/Q19652" class="extiw" title="u200epublic domainu200e | u200eno exclusive intellectual property rights applyu200e"><span class="wb-itemlink"><span class="wb-itemlink-label" lang="en" dir="ltr">public domain</span> <span class="wb-itemlink-id">(Q19652)</span></span></a>, updating based on <a href="/wiki/Template:PD-USGov-NPS" title="Template:PD-USGov-NPS">Template:PD-USGov-NPS</a>"}


file_path = "/home/gauravthagunna/Desktop/projects/kafka-projects/wikimedia_processing/data_sample/data_wikimedia_test.json"

def getSchema():
    return """schema string , meta struct <uri string, request_id string, id string, dt string, partition string >,
    id bigint,title string, bot boolean, notify_url string, length struct <old string, new string>,wiki string
        """
def getSchema2():
    return """schema string """
df = (spark.read
                    .format("json")
                    .schema(getSchema())
                    .load(f"{file_path}")
                )   
df.show(truncate=False)

+------+----+----+-----+----+----------+------+----+
|schema|meta|id  |title|bot |notify_url|length|wiki|
+------+----+----+-----+----+----------+------+----+
|null  |null|null|null |null|null      |null  |null|
+------+----+----+-----+----+----------+------+----+



In [173]:

#WORKING VERSION 

# Replace escaped backslashes with single ones to ensure correct parsing
df = df.withColumn('value',F.col("value").cast("string"))
parsed_data = df.withColumn("value_clean", F.regexp_replace("value", r'\\\\', r'\\'))
# Remove backslashes from the start of key names and values, as well as the quotes at the start and end
parsed_data = parsed_data.withColumn(
    "value_clean",
    F.regexp_replace(F.col("value_clean"), r'\\', '')
)

# Also remove double quotes at the beginning and end if not done already
parsed_data = parsed_data.withColumn(
    "value_clean",
    F.regexp_replace(F.col("value_clean"), r'^"|"$', '')
)
parsed_data = parsed_data.withColumn("value_clean", F.regexp_replace("value_clean", r"\$", ""))


simple_schema = """schema string , meta struct <uri string, request_id string, id string, dt string, domain string, stream string >,
                    id bigint, type string, namespace string, title string, user string, bot string, minor boolean, pantrolled boolean,
                    length struct < old int, new int >,revision struct < old bigint, new bigint>,server_name string, wiki string
"""


df_parsed = parsed_data.withColumn("data", F.from_json(F.col("value_clean"), simple_schema))
df_parsed = (df_parsed.select("data.schema", "data.meta.uri", 
                              "data.meta.request_id","data.meta.id","data.meta.dt","data.meta.domain","data.meta.stream",
                              "data.id","data.type","data.namespace","data.title","data.user","data.bot","data.minor","data.pantrolled","data.length.old","data.length.new",
                              "data.revision.old","data.revision.new","data.server_name","data.wiki"
                              
                              ))
# df_parsed = df_parsed.withColumnRenamed("json_data.$schema","schema").withColumnRenamed("json_data.type","type").withColumnRenamed("json_data.id","id")
df_parsed.show()

# parsed_data.show()

24/11/15 18:20:50 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 18:20:50 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 18:20:50 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 18:20:50 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 18:20:50 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+---------+----+---------+--------------------+-------------+-----+-----+----------+------+------+---------+---------+-----------------+------------+
|              schema|                 uri|          request_id|                  id|                  dt|           domain|              stream|       id|type|namespace|               title|         user|  bot|minor|pantrolled|   old|   new|      old|      new|      server_name|        wiki|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+---------+----+---------+--------------------+-------------+-----+-----+----------+------+------+---------+---------+-----------------+------------+
|                null|                null|                null|                null|                null|            

In [163]:
df_parsed.count()

24/11/15 17:48:08 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 17:48:08 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 17:48:08 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 17:48:08 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 17:48:08 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


182

In [166]:
df_parsed.filter(F.col("id").isNotNull()).count()

24/11/15 17:48:35 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 17:48:35 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 17:48:35 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 17:48:35 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 17:48:35 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


30

In [54]:
from pyspark.sql import functions as F

from pyspark.sql import functions as F

# Remove backslashes from the start of key names and values, as well as the quotes at the start and end
parsed_data = parsed_data.withColumn(
    "value_clean",
    F.regexp_replace(F.col("value_clean"), r'\\', '')
)

# Also remove double quotes at the beginning and end if not done already
parsed_data = parsed_data.withColumn(
    "value_clean",
    F.regexp_replace(F.col("value_clean"), r'^"|"$', '')
)

# Verify the output
parsed_data.select("value_clean").show(truncate=False)



24/11/15 13:54:22 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 13:54:22 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 13:54:22 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 13:54:22 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 13:54:22 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
json_string = parsed_data.select("value_clean").toJSON().collect()[0] 
  
print(json_string) 
  
# Write the JSON string to a file 
with open("/home/gauravthagunna/Desktop/projects/kafka-projects/wikimedia_processing/data_sample/data_wikimedia.json", "w") as f: 
    f.write(json_string) 

24/11/15 13:59:54 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 13:59:54 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 13:59:54 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 13:59:54 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 13:59:54 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


{"value_clean":"\"{\\\"$schema\\\":\\\"/mediawiki/recentchange/1.0.0\\\",\\\"meta\\\":{\\\"uri\\\":\\\"https://commons.wikimedia.org/wiki/File:Florida_Tropical_House_NPS.jpg\\\",\\\"request_id\\\":\\\"0c4303be-f77f-4c51-8691-9ff1571c6298\\\",\\\"id\\\":\\\"56cf14b8-fb3e-4c2d-b96b-25e88060b96d\\\",\\\"dt\\\":\\\"2024-11-15T07:12:58Z\\\",\\\"domain\\\":\\\"commons.wikimedia.org\\\",\\\"stream\\\":\\\"mediawiki.recentchange\\\",\\\"topic\\\":\\\"codfw.mediawiki.recentchange\\\",\\\"partition\\\":0,\\\"offset\\\":1249027121},\\\"id\\\":2655168092,\\\"type\\\":\\\"edit\\\",\\\"namespace\\\":6,\\\"title\\\":\\\"File:Florida Tropical House NPS.jpg\\\",\\\"title_url\\\":\\\"https://commons.wikimedia.org/wiki/File:Florida_Tropical_House_NPS.jpg\\\",\\\"comment\\\":\\\"/* wbsetclaim-update:2||1|2 */ [[d:Special:EntityPage/P6216]]: [[d:Special:EntityPage/Q19652]], updating based on [[Template:PD-USGov-NPS]]\\\",\\\"timestamp\\\":1731654778,\\\"user\\\":\\\"BotMultichillT\\\",\\\"bot\\\":true,\\\"

In [31]:
df.count().sh

24/11/15 13:29:06 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/11/15 13:29:06 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/11/15 13:29:06 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/11/15 13:29:06 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/11/15 13:29:06 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


182